In [7]:
import os
os.chdir('..')  # Change directory to the parent folder

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen=True)
class EvaluateConfig:
    path_of_model: Path
    training_data: Path
    all_param: dict
    mlflow_uri: str
    param_image_size: list
    param_batch_size: int

In [26]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluateConfig:
        eval_config=EvaluateConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(self.config.data_ingestion.unzip_dir,"Chest-CT-Scan-data"),
            all_param=self.params,
            mlflow_uri='https://dagshub.com/cleavestone94/ChestXray-Cancer-Detection.mlflow',
            param_image_size=self.params.IMAGE_SIZE,
            param_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [34]:
from cnnClassifier.utils.common import save_json
import dagshub
dagshub.init(repo_owner='cleavestone94', repo_name='ChestXray-Cancer-Detection', mlflow=True)

class Evaluation:
    def __init__(self,config: EvaluateConfig):
        self.config=config

    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.3
        )

        dataflow_kwargs=dict(
            target_size=self.config.param_image_size[:-1],
            batch_size=self.config.param_batch_size,
            interpolation="bilinear",
        )

        valid_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

        
    def save_score(self):
        scores={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_param)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model,"model")
    

{2025-08-22 14:00:35,043: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/cleavestone94/ChestXray-Cancer-Detection "HTTP/1.1 200 OK"}


Initialized MLflow to track repo "cleavestone94/ChestXray-Cancer-Detection"

{2025-08-22 14:00:35,086: INFO: helpers: Initialized MLflow to track repo "cleavestone94/ChestXray-Cancer-Detection"}


Repository cleavestone94/ChestXray-Cancer-Detection initialized!

{2025-08-22 14:00:35,097: INFO: helpers: Repository cleavestone94/ChestXray-Cancer-Detection initialized!}


In [20]:
import os
os.chdir('..')  # Change directory to the parent folder
# check if the current working directory is the project root
print(f"Current working directory: {os.getcwd()}")
# change dir to C:\Users\Hp\Desktop\Chest_Cancer_Classification
os.chdir('C:/Users/Hp/Desktop/Chest_Cancer_Classification')


Current working directory: c:\


In [21]:
os.getcwd()

'C:\\Users\\Hp\\Desktop\\Chest_Cancer_Classification'

In [35]:
try:
    config=ConfigurationManager()
    eval_config=config.get_evaluation_config()
    evaluation=Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

{2025-08-22 14:00:50,236: INFO: common: yaml file: config\config.yaml loaded successfully}
{2025-08-22 14:00:50,244: INFO: common: yaml file: params.yaml loaded successfully}
{2025-08-22 14:00:50,248: INFO: common: created directory at: artifacts_root}
Found 102 images belonging to 2 classes.
7/7 [==============================] - 33s 5s/step - loss: 0.6551 - accuracy: 0.5686
{2025-08-22 14:01:25,081: INFO: common: json file saved at: scores.json}


2025/08/22 14:01:26 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


{2025-08-22 14:01:29,301: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.}
{2025-08-22 14:01:31,505: INFO: builder_impl: Assets written to: C:\Users\Hp\AppData\Local\Temp\tmpec_ed57t\model\data\model\assets}


c:\Users\Hp\Desktop\Chest_Cancer_Classification\venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/08/22 14:04:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
